In [65]:
import requests
import json
from datetime import datetime, date
from typing import Dict, Any
from dateutil.parser import isoparse


In [ ]:
BASE_URL = "http://127.0.0.1:8000"
HEADERS = {"Content-Type": "application/json"}


In [67]:
# 1. Index Endpoint
try:
    response = requests.get(f"{BASE_URL}/", timeout=10)
    response.raise_for_status()
    print("Index Response:", response.json())
except requests.RequestException as e:
    print(f"Error accessing index endpoint: {str(e)}")


Index Response: Server is up and running


In [68]:
# 2. Health Endpoint
try:
    response = requests.get(f"{BASE_URL}/health", timeout=10)
    response.raise_for_status()
    print("Health Response:", response.json())
except requests.RequestException as e:
    print(f"Error accessing health endpoint: {str(e)}")


Health Response: {'status': 'healthy'}


In [69]:
# Load and preprocess transaction history
try:
    with open('transaction_history.json', 'r') as file:
        transactions = json.load(file)

    def convert_to_float(value):
        if isinstance(value, str):
            return float(value.replace(',', ''))
        return value  # If it's already a number, leave it unchanged

    def filter_transactions_by_date(transactions, start_date=None, end_date=None):
        if not start_date and not end_date:
            return transactions
        filtered = []
        start = isoparse(start_date).replace(tzinfo=None) if start_date else None
        end = isoparse(end_date).replace(tzinfo=None) if end_date else None
        for t in transactions:
            transaction_date = isoparse(t['transactionDate']).replace(tzinfo=None)
            if start and end:
                if start <= transaction_date <= end:
                    filtered.append(t)
            elif start and transaction_date >= start:
                filtered.append(t)
            elif end and transaction_date <= end:
                filtered.append(t)
        return filtered

    for transaction in transactions:
        transaction['amount'] = convert_to_float(transaction.get('amount'))
        transaction['balance'] = convert_to_float(transaction.get('balance'))
        # Ensure transactionDate is a string in ISO 8601 format
        if isinstance(transaction['transactionDate'], datetime):
            transaction['transactionDate'] = transaction['transactionDate'].isoformat()
except FileNotFoundError:
    print("Error: transaction_history.json not found")
except json.JSONDecodeError:
    print("Error: Invalid JSON in transaction_history.json")


In [70]:
# Query: Number of debit transactions
prompt = "How many debit transactions do I have"
get_audio = False
thread_id = "1"

endpoint = f"{BASE_URL}/conversation"
payload = {
    "prompt": prompt,
    "transactions": transactions,  # Send all transactions
    "thread_id": thread_id,
    "get_audio": get_audio
}
try:
    response = requests.post(endpoint, json=payload, headers=HEADERS, timeout=120)
    response.raise_for_status()
    result = response.json()
    print("Debit Transactions Response:", result)
except requests.RequestException as e:
    print(f"Error querying debit transactions: {str(e)}")


Debit Transactions Response: {'messages': 'You have a total of 37 debit transactions.', 'audio': None}


In [71]:
# Query: Number of expense transactions in January 2025
prompt = "How many expense transactions do I have in January 2025"
get_audio = False
thread_id = "1"

endpoint = f"{BASE_URL}/conversation"
payload = {
    "prompt": prompt,
    "transactions": transactions,  # Send filtered transactions
    "thread_id": thread_id,
    "get_audio": get_audio
}
try:
    response = requests.post(endpoint, json=payload, headers=HEADERS, timeout=120)
    response.raise_for_status()
    result = response.json()
    print("Expense Transactions Response:", result)
except requests.RequestException as e:
    print(f"Error querying expense transactions: {str(e)}")


Expense Transactions Response: {'messages': 'You have a total of 72 expense transactions in January 2025.', 'audio': None}


In [72]:
# Query: Total income in January 2025
prompt = "What is my total income in January 2025"
get_audio = False
thread_id = "1"

endpoint = f"{BASE_URL}/conversation"
payload = {
    "prompt": prompt,
    "transactions": transactions,  # Send filtered transactions
    "thread_id": thread_id,
    "get_audio": get_audio
}
try:
    response = requests.post(endpoint, json=payload, headers=HEADERS, timeout=120)
    response.raise_for_status()
    result = response.json()
    print("Total Income Response:", result)
except requests.RequestException as e:
    print(f"Error querying total income: {str(e)}")


Total Income Response: {'messages': 'Your total income in January 2025 is 325,931,182.00 NGN.', 'audio': None}
